# Beaking down the code

In [1]:
%load_ext autoreload
%autoreload 2

## The `Downloader` Class

Now that we have a class to download the files and the parsers to understand the remote structure, let's take a look at the `Downloader` class, that combines both structures

In [2]:
import logging
from datetime import datetime
from mergedownloader.inpeparser import *
from mergedownloader.file_downloader import FileDownloader, ConnectionType, DownloadMode
from mergedownloader.downloader import Downloader


In [3]:
fd = FileDownloader(server=INPE_SERVER, connection_type=ConnectionType.HTTP, download_mode=DownloadMode.UPDATE)

Using wget through HTTP on: ftp.cptec.inpe.br


In [4]:
downloader = Downloader(
    file_downloader=fd,
    parsers=InpeParsers,
    local_folder='/tmp2',
    log_level=logging.DEBUG
)

In [5]:
# Let's test downloading one file
p = downloader.get_parser('DAILY_AVERAGE')
print(p.remote_target(date=datetime(2022, 1, 1)))
print(p.local_folder(date=datetime(2022, 1, 1), output_folder='/tmp2'))

/modelos/tempo/MERGE/GPM/CLIMATOLOGY/DAILY_AVERAGE/MERGE_CPTEC_12Z01jan.nc
/tmp2/DAILY_AVERAGE


In [6]:
file = downloader.get_file(date='2022-01-01', datatype='DAILY_AVERAGE')
assert file.exists()
file

PosixPath('/tmp2/DAILY_AVERAGE/MERGE_CPTEC_12Z01jan.nc')

In [7]:
downloader.get_files(dates=['2022-01-01', '2022-01-02', '1999-01-01'], datatype='DAILY_RAIN')

[PosixPath('/tmp2/DAILY/2022/01/MERGE_CPTEC_20220101.grib2'),
 PosixPath('/tmp2/DAILY/2022/01/MERGE_CPTEC_20220102.grib2'),
 None]

In [8]:
downloader.get_range(start_date='2022-01-02', end_date='2022-02-05', datatype=InpeTypes.MONTHLY_ACCUM)

[PosixPath('/tmp2/MONTHLY_ACCUMULATED/MERGE_CPTEC_acum_jan.nc'),
 PosixPath('/tmp2/MONTHLY_ACCUMULATED/MERGE_CPTEC_acum_feb.nc')]

In [9]:
ds = downloader.open_file(date='2022-01-01', datatype='DAILY_AVERAGE')
ds

/usr/local/lib/python3.10/dist-packages/ecmwflibs/__init__.py:81: UserWarning: libglib-2.0.so.0: cannot open shared object file: No such file or directory
  warnings.warn(str(e))


<xarray.DataArray 'pmed' (time: 1, lat: 691, lon: 551)>
[380741 values with dtype=float64]
Coordinates:
  * lon      (lon) float64 -85.05 -84.95 -84.85 -84.75 ... -30.25 -30.15 -30.05
  * lat      (lat) float64 -56.15 -56.05 -55.95 -55.85 ... 12.65 12.75 12.85
  * time     (time) datetime64[ns] 2020-01-01T12:00:00

In [13]:
arr = downloader.open_file(date='2022-01-01', datatype='DAILY_RAIN')
arr

<xarray.DataArray 'prec' (latitude: 924, longitude: 1001)>
[924924 values with dtype=float32]
Coordinates:
    time         datetime64[ns] ...
    step         timedelta64[ns] ...
    surface      float64 ...
  * latitude     (latitude) float64 -60.05 -59.95 -59.85 ... 32.05 32.15 32.25
  * longitude    (longitude) float64 -120.1 -120.0 -119.9 ... -20.15 -20.05
    valid_time   datetime64[ns] ...
    spatial_ref  int64 0
Attributes: (12/29)
    GRIB_paramId:                             260138
    GRIB_dataType:                            an
    GRIB_numberOfPoints:                      924924
    GRIB_typeOfLevel:                         surface
    GRIB_stepUnits:                           1
    GRIB_stepType:                            instant
    ...                                       ...
    GRIB_name:                                Precipitation
    GRIB_shortName:                           prec
    GRIB_units:                               kg m**-2
    long_name:                                Precipitation
    units:                                    kg m**-2
    standard_name:                            unknown